# PREPROCESSING ON THE SAMPLED_DATASET

In [255]:
import pandas as pd
df = pd.read_csv("sampled_dataset.csv")
df.head()

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country
2,Friends With The Enemy,Poor Man's Poison,2011,5611,"{""Poor Man\\'s Poison""}",[Verse 1]\nLonging for something more\nWill I ...,True,False,False,False,False,is_country
3,Sleepin With the Radio On,Charly McClain,1981,100,{},My heart beats with the lonely rain\nWishin' I...,True,False,False,False,False,is_country
4,Riders in the Sky,Michael Martin Murphey,1993,108,{},An old cowpoke went ridin' out one dark and wi...,True,False,False,False,False,is_country


In [256]:
df.tail()

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag
161655,From Dust to Dust,Horcrvx,2018,23,{},Six feet beneath the creek bed; my soul does f...,False,False,False,False,True,is_rock
161656,​cutting emptiness,Sadness (USA),2021,12,{},Cutting emptiness as though it will fill me\nA...,False,False,False,False,True,is_rock
161657,Coasting,Ten F,2019,867,"{""Ten Fé""}",[Verse 1]\nThe night is warmer now\nI want to ...,False,False,False,False,True,is_rock
161658,Plunderberg,Lagerstein,2012,3,{},"We loot, we steal, we Plunderberg\nSailing thr...",False,False,False,False,True,is_rock
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock


### PULIZIA LYRICS

In [257]:
#ripulire lyrics tenendo solo tag importanti per divisione in strofe

import re
keep_pattern = r"\[(Chorus|Verse|Bridge|Intro|Outro|Hook|Prehook|Posthook|Introduction|Interlude|Coda|Conclusion|Refrain).\]*"

# Function to remove undesired bracketed text
def clean_lyrics(lyrics):
    # Remove any text inside brackets that isn't a recognized section
    cleaned_lyrics = re.sub(r"\[.*?\]", lambda match: match.group(0) if re.match(keep_pattern, match.group(0)) else "", lyrics)
    return cleaned_lyrics

# Example usage with a DataFrame
df['cleaned_lyrics'] = df['lyrics'].apply(clean_lyrics)

In [258]:
#function to split the stanzas with different criterion   

def process_lyric(ly):
    sections = [ly.strip()]
    
    # First split by the pattern
    pattern = r"\[(Introduction|Interlude|Coda|Hook|Prehook|Posthook|Conclusion|Refrain|Verse|Intro|Outro|Chorus|Bridge).\]*"
    if re.search(pattern, ly, re.IGNORECASE):
        sections = [segment for section in sections for segment in section.split("\n[")]

    # Further split by "Verse"
    if re.search(r"Verse", ly, re.IGNORECASE):
        sections = [segment for section in sections for segment in section.split("Verse")]

    # Further split by double newlines
    if "\n\n" in ly:
        sections = [segment for section in sections for segment in section.split("\n\n")]
    
    # Further split by "(Chorus)"
    if re.search(r"\(Chorus\)", ly, re.IGNORECASE):
        sections = [segment for section in sections for segment in section.split("(")]

    # Clean up whitespace for each section and remove empty sections
    return [section.strip() for section in sections if section.strip()]

# Apply the function to the cleaned_lyrics column
df["processed_lyrics"] = df["cleaned_lyrics"].apply(process_lyric)

In [259]:
#function to check if nella lista di stringhe ci sono delle stringhe con solo "hook", "chorus" ecc. e stringhe vuote e in caso eliminarle
#"x3 "
#]
#(repeat x3)
def clean_list(lyrics_list): 
    if not isinstance(lyrics_list, list):
        return lyrics_list
    
    # Regex pattern to match tags like "Hook]", "Chorus]", etc.
    tag_pattern = r"^\s*(Hook|Chorus|Bridge|Verse|Outro|Intro|Refrain|Prehook|Posthook|Coda|Interlude|Conclusion).*?\]\s*$|^(.{,20})\s*$"
    #tag_pattern = r"^\s*(Hook|Chorus|Bridge|Verse|Outro|Intro|Refrain|Prehook|Posthook|Coda|Interlude|Conclusion)\s*.*?\]\s*$"
    #tag_pattern = r"^\s*(Hook|Chorus|Bridge|Verse|Outro|Intro|Refrain|Prehook|Posthook|Coda|Interlude|Conclusion)\s*.*?\]?\s*$" questo mette la parentesi in fondo opzionale
    
    # Filter out empty strings and strings matching the tag pattern
    return [line for line in lyrics_list if line and not re.match(tag_pattern, line, re.IGNORECASE)]


# Apply the cleaning function to the specified column
df['cleaned_lists'] = df["processed_lyrics"].apply(clean_list)

In [260]:
df.head()

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag,cleaned_lyrics,processed_lyrics,cleaned_lists
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She got a broke down El Camino, in the front y...","[She got a broke down El Camino, in the front ...","[She got a broke down El Camino, in the front ..."
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Later on\nWhen we've all come down\nAnd the st...,[Later on\nWhen we've all come down\nAnd the s...,[Later on\nWhen we've all come down\nAnd the s...
2,Friends With The Enemy,Poor Man's Poison,2011,5611,"{""Poor Man\\'s Poison""}",[Verse 1]\nLonging for something more\nWill I ...,True,False,False,False,False,is_country,[Verse 1]\nLonging for something more\nWill I ...,"[[, 1]\nLonging for something more\nWill I eve...",[1]\nLonging for something more\nWill I ever b...
3,Sleepin With the Radio On,Charly McClain,1981,100,{},My heart beats with the lonely rain\nWishin' I...,True,False,False,False,False,is_country,My heart beats with the lonely rain\nWishin' I...,[My heart beats with the lonely rain\nWishin' ...,[My heart beats with the lonely rain\nWishin' ...
4,Riders in the Sky,Michael Martin Murphey,1993,108,{},An old cowpoke went ridin' out one dark and wi...,True,False,False,False,False,is_country,An old cowpoke went ridin' out one dark and wi...,[An old cowpoke went ridin' out one dark and w...,[An old cowpoke went ridin' out one dark and w...


### CONTROLLO SU PULIZIA LYRICS

In [261]:
x = df.loc[0, "processed_lyrics"]
x

['She got a broke down El Camino, in the front yard up on blocks\nHer mom walks around in a pink nightgown, sandals and white socks\nShe don’t mind a baseball game in the middle of the lightning and the rain\nShe’s a pain in my brain, drives me insane\nBut I love her just the same, boys, love her just the same',
 'Chorus]\nSomething in the water\n(something in the water)\nSomething in the water\n(something in the water)\nOoh-ooh-ooh-ooh\nWhat makes her crazy I don’t know\nNever seen anything like her before\nThere must be something in the water',
 'She does her makeup and hair, to cook fried chicken in her underwear\nShe drinks malt liquor for lunch and dinner and sends me running scared\nShe yells, she screams and she beats me\nBut I don’t mind the way she treats me\nShe’ll someday lead to my death I know\nBut I’ll stay with her just the same, boys, stay with her just the same',
 'Chorus]\nSomething in the water\n(something in the water)\nSomething in the water\n(something in the wate

In [262]:
x = df.loc[1, ["title", "artist"]]
x

title     The Last Event
artist      Matthew Ryan
Name: 1, dtype: object

In [263]:
x = df.loc[1, "cleaned_lists"]
x

["Later on\nWhen we've all come down\nAnd the streets are just a funeral\nAnd the last event\nIs\u2005falling\u2005ash and embers\nWe'll\u2005drift and waltz and twist into\u2005nothing\nWe weren't too bright my love, but\nWeren't we something?",
 'Heroes waved\nFrom\u205frented\u205fblack\u205fsedans\nMonsters crept behind\u205ftall buildings\nAnd we\u205fsmiled and cheered\nLike nothing ever ended\nWe were so beautiful we forgot we were human\nIn dusk and fear amen\nWe found our undoing',
 "Don't say it comes as any shock\nThings just go and go and go until they stop\nI'll always remember you as a song with no end\nHere until...\nEarly May\nShe was standing in the front yard\nThe sun was perfect, the clouds gone missing\nAnd I couldn't move\nThat's when she started crying\nIt felt so quick and yet somehow in slow motion\nWe're all just boats that sink while falling in love\nWith the ocean",
 "Don't say it comes as any shock\nThings just go and go and go until they stop\nI'll always r

### EXPLODE SAMPLED_DATA

In [264]:
#removing redundant variables
df = df.drop(['cleaned_lyrics', 'processed_lyrics'], axis=1)

In [ ]:
exploded_df = df.explode('cleaned_lists', ignore_index=False)
exploded_df.rename(columns={'cleaned_lists': 'stanzas'}, inplace=True)

# numbers the stanzas based on their order
exploded_df['stanza_number'] = exploded_df.groupby(exploded_df.index).cumcount()


exploded_df

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag,stanzas
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She got a broke down El Camino, in the front y..."
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Chorus]\nSomething in the water\n(something in...
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She does her makeup and hair, to cook fried ch..."
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Chorus]\nSomething in the water\n(something in...
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"My hoosier girl is so fine, shake the watermel..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,[Refrain]\nThe thin gruel for millions of people
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,1]\nNothing is left but a mountain of shoes\nD...
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,Refrain]\nThe thin gruel for millions of people
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,2]\nSometimes you live\nAnd sometimes you die\...


In [266]:
for v in exploded_df["stanzas"]:
    print(type(v))
    break

<class 'str'>


In [ ]:
# SOLVED
#il problema è che non considera i valori di "stanzas" come stringhe 
#ma strano perchè sopra dice di sì

def is_chorus(stanza):
    if isinstance(stanza, str):
        pattern = r"^\s*(Chorus)\s*(\]|\))"
        return bool(re.match(pattern, stanza, re.IGNORECASE))

#!TOCHECK
# this checks for is_chorus and whether a stanza is repeated
# for the same song (index)
exploded_df["is_chorus"] = (
    exploded_df["stanzas"].apply(is_chorus) | 
    exploded_df.groupby(exploded_df.index)["stanzas"].transform(
        lambda x: x.duplicated(keep=False)
    )
)

In [268]:
exploded_df.head(10)

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag,stanzas,is_chorus
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She got a broke down El Camino, in the front y...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Chorus]\nSomething in the water\n(something in...,True
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She does her makeup and hair, to cook fried ch...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Chorus]\nSomething in the water\n(something in...,True
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"My hoosier girl is so fine, shake the watermel...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Chorus]\nSomething in the water\n(something in...,True
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Later on\nWhen we've all come down\nAnd the st...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Heroes waved\nFrom rented black sedans\nMonste...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Don't say it comes as any shock\nThings just g...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Don't say it comes as any shock\nThings just g...,False


In [269]:
exploded_df.tail()

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag,stanzas,is_chorus
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,[Refrain]\nThe thin gruel for millions of people,False
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,1]\nNothing is left but a mountain of shoes\nD...,False
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,Refrain]\nThe thin gruel for millions of people,False
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,2]\nSometimes you live\nAnd sometimes you die\...,False
161659,The Thin Gruel,Buck Gooter,2012,28,{},[Refrain]\nThe thin gruel for millions of peop...,False,False,False,False,True,is_rock,Refrain]\nThe thin gruel for millions of people,False


In [270]:
#numero totale di righe
len(exploded_df)

966423

In [271]:
duplicate_count = exploded_df.duplicated().sum()
print(f"Total number of duplicate rows: {duplicate_count}")

Total number of duplicate rows: 91579


In [272]:
exploded_df = exploded_df.drop_duplicates()

In [273]:
duplicate_count = exploded_df.duplicated().sum()
print(f"Total number of duplicate rows: {duplicate_count}")

Total number of duplicate rows: 0


In [279]:
x = exploded_df.loc[2, "stanzas"]
x

2     Longing for something more Will I ever be fre...
2     Workin’ on a better way Will I ever be free? ...
2     And them hard times keep comin’ at the worst ...
2     Never more will you see my face Will I ever b...
2     And them hard times keep comin’ at the worst ...
2     And them hard times keep comin’ at the worst ...
Name: stanzas, dtype: object

In [275]:
exploded_df.head(10)

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag,stanzas,is_chorus
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She got a broke down El Camino, in the front y...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Chorus]\nSomething in the water\n(something in...,True
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She does her makeup and hair, to cook fried ch...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"My hoosier girl is so fine, shake the watermel...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Chorus]\nSomething in the water\n(something in...,True
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Later on\nWhen we've all come down\nAnd the st...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Heroes waved\nFrom rented black sedans\nMonste...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Don't say it comes as any shock\nThings just g...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Don't say it comes as any shock\nThings just g...,False
2,Friends With The Enemy,Poor Man's Poison,2011,5611,"{""Poor Man\\'s Poison""}",[Verse 1]\nLonging for something more\nWill I ...,True,False,False,False,False,is_country,1]\nLonging for something more\nWill I ever be...,False


In [276]:
count = 0
for i in exploded_df["is_chorus"]:
    if i == True:
        count += 1
count

63677

In [277]:
def pulire_stringhe(stanza):
    # Define the pattern
    pattern = r"^.*?\]|\)\n*"
    
    # Apply the regex substitution to clean the stanza
    result1 = re.sub(pattern, "", stanza)
    result2 = re.sub(r"\n", " ", result1)
    
    return result2

# Apply the function to the 'stanzas' column
exploded_df["stanzas"] = exploded_df["stanzas"].apply(pulire_stringhe)


C:\Users\Gagia\AppData\Local\Temp\ipykernel_18372\2710612599.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exploded_df["stanzas"] = exploded_df["stanzas"].apply(pulire_stringhe)


In [278]:
exploded_df.head(15)

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag,stanzas,is_chorus
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She got a broke down El Camino, in the front y...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Something in the water (something in the wate...,True
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She does her makeup and hair, to cook fried ch...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"My hoosier girl is so fine, shake the watermel...",False
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Something in the water (something in the wate...,True
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Later on When we've all come down And the stre...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Heroes waved From rented black sedans Monsters...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Don't say it comes as any shock Things just go...,False
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Don't say it comes as any shock Things just go...,False
2,Friends With The Enemy,Poor Man's Poison,2011,5611,"{""Poor Man\\'s Poison""}",[Verse 1]\nLonging for something more\nWill I ...,True,False,False,False,False,is_country,Longing for something more Will I ever be fre...,False


- raffinare regex (FATTO)
- controllare se non stiamo togliendo troppo (FATTO)
- fare explode (FATTO)
- controllare se stringa è chorus o no (FATTO)
- pulire stringhe da intestazioni di strofa (FATTO)
- tokenizzare, stopwords removal, lemmatization

In [285]:
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser", "textcat"])


In [ ]:
def tokenize(stanza):
    # Process the text with spaCy
    if isinstance(stanza, str):
        stanza = stanza.lower()
        doc = nlp(stanza)
        tokens = [token.lemma_ for token in doc if token != ""]
        return tokens

  
exploded_df["lemmatized_stanzas"] = exploded_df["stanzas"].apply(tokenize)
exploded_df.head()

C:\Users\Gagia\AppData\Local\Temp\ipykernel_18372\3666439909.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exploded_df["lemmatized_stanzas"] = exploded_df["stanzas"].apply(tokenize)


,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag,stanzas,is_chorus,lemmatized_stanzas
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She got a broke down El Camino, in the front y...",False,"[she, get, a, broke, down, El, Camino, ,, in, ..."
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Something in the water (something in the wate...,True,"[ , something, in, the, water, (, something, i..."
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She does her makeup and hair, to cook fried ch...",False,"[she, do, her, makeup, and, hair, ,, to, cook,..."
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"My hoosier girl is so fine, shake the watermel...",False,"[my, hoosi, girl, be, so, fine, ,, shake, the,..."
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,Something in the water (something in the wate...,True,"[ , something, in, the, water, (, something, i..."


In [287]:
x = exploded_df.loc[0, "lemmatized_stanzas"]
x

0    [she, get, a, broke, down, El, Camino, ,, in, ...
0    [ , something, in, the, water, (, something, i...
0    [she, do, her, makeup, and, hair, ,, to, cook,...
0    [my, hoosi, girl, be, so, fine, ,, shake, the,...
0    [ , something, in, the, water, (, something, i...
Name: lemmatized_stanzas, dtype: object

## cose da valutare:
- il rumore nelle varie strofe (intercalari, "yeaaah", "ohh-ohh" ecc.)
- lemmatizzazione o stemming?
- questione di: ,,
- lowercasing (che pensavamo facesse in automatico)

### per gli ultimi due punti ho aggiornato la funzione ma non l'ho runnata

In [ ]:
#exploded_df = exploded_df.drop("lyrics", axis=1)
#exploded_df.to_csv('first_preprocessed_df.csv', index=False)